# Car Damage Assessment
Instance segmentation for:
1. Damage localization (COCO_train_annos.json)
2. Parts localization (COCO_mul_train_annos.json)

Output:
- Assert is photo a car?
- Match detected damage to detected part

In [5]:
# Build Detectron2 from Source

!python -m pip install "git+https://github.com/facebookresearch/detectron2.git"

  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\i17215834\appdata\local\temp\pip-req-build-nr85edku
  Resolved https://github.com/facebookresearch/detectron2.git to commit 224cd2318fdb45b5e22bbb861ee9711ee52c8b75
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for detectron2
Failed to build detectron2
  Running setup.py install for detectron2: started
  Running setup.py install for detectron2: finished with status 'error'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git 'C:\Users\I17215834\AppData\Local\Temp\pip-req-build-nr85edku'
  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [3772 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\detectron2
  copying detectron2\__init__.py -> build\lib.win-amd64-3.8\detectron2
  creating build\lib.win-amd64-3.8\tools
  copying tools\analyze_model.py -> build\lib.win-amd64-3.8\tools
  copying tools\benchmark.py -> build\lib.win-amd64-3.8\tools
  copying tools\convert-torchvision-to-d2.py -> build\lib.win-amd64-3.8\tools
  copying tools\lazyconfig_train_net.py -> build\lib.win-amd64-3.8\tools
  copying tools\lightning_train_net.py -> build\lib.win-amd64-3.8\tools
  copying tools\plain_train_net.py -> build\lib.win-amd64-3.8\tools
  cop

In [6]:
import detectron2
import torch

!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Mar_21_19:24:09_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.3, V11.3.58
Build cuda_11.3.r11.3/compiler.29745058_0
torch:  1.10 ; cuda:  cu113


NameError: name 'detectron2' is not defined

In [2]:
# import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

import json
import os
import random
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import (
    DatasetCatalog,
    DatasetMapper,
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.data import transforms as T
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.utils.visualizer import ColorMode, Visualizer

# Set base params
plt.rcParams["figure.figsize"] = [16, 9]

ModuleNotFoundError: No module named 'detectron2'

## Dataset

In [ ]:
root_path = Path().absolute().parents[0]
data_path = root_path / "data"

train_path = data_path / "train"
train_ann_path = train_path / "COCO_train_annos.json"
train_parts_ann_path = train_path / "COCO_mul_train_annos.json"

val_path = data_path / "val"
val_ann_path = val_path / "COCO_val_annos.json"
val_parts_ann_path = val_path / "COCO_mul_val_annos.json"

test_path = data_path / "test"

In [ ]:
multi_class_parts_categories = [
    "headlamp",
    "rear_bumper",
    "door",
    "hood",
    "front_bumper",
]

In [ ]:
register_coco_instances("car_dataset_train", {}, train_ann_path, train_path)
register_coco_instances("car_dataset_val", {}, val_ann_path, val_path)

In [ ]:
dataset_dicts = DatasetCatalog.get("car_dataset_train")
metadata_dicts = MetadataCatalog.get("car_dataset_train")

## Augmentations

In [ ]:
# Define a sequence of augmentations:
augs = T.AugmentationList(
    [
        T.RandomBrightness(0.9, 1.1),
        T.RandomFlip(prob=0.5),
        # T.RandomCrop("absolute", (640, 640))
    ]
)  # type: T.Augmentation

## Train

In [ ]:
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):

        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"

        return COCOEvaluator(dataset_name, cfg, False, output_folder)

    @classmethod
    def build_train_loader(cls, cfg):

        augmentations = augs

        return build_detection_train_loader(
            cfg,
            mapper=DatasetMapper(cfg, is_train=True, augmentations=augmentations),
        )

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
)
cfg.DATASETS.TRAIN = ("car_dataset_train",)
cfg.DATASETS.TEST = ("car_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.MODEL.RETINANET.NUM_CLASSES = 2
cfg.TEST.EVAL_PERIOD = 600

# Clear any logs from previous runs
!rm -rf cfg.OUTPUT_DIR

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

## Inference & Evaluation
using the trained model

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(
    cfg.OUTPUT_DIR, "model_final.pth"
)  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluator = COCOEvaluator("car_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "balloon_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`